In [2]:
import pandas as pd
import re
from typing import List

class CourseFilter:
    """Класс для фильтрации курсов по заданным названиям"""
    
    def __init__(self, filter_names: List[str]):
        # Компилируем regex один раз для производительности
        self.normalize_pattern = re.compile(r'[^\w\s\-"]')
        self.whitespace_pattern = re.compile(r'\s+')
        self.lesson_pattern = re.compile(r'\s+\-\s+занятие\s+\d+.*')
        self.group_pattern = re.compile(r'\s+группа\s+\d+.*')
        
        # Предварительно нормализуем фильтры
        self.normalized_filters = [self._normalize_text(name) for name in filter_names]
    
    def _normalize_text(self, text: str) -> str:
        """Нормализация текста с использованием предварительно скомпилированных regex"""
        if not isinstance(text, str):
            return ""
        
        normalized = text.lower().strip()
        normalized = self.normalize_pattern.sub('', normalized)
        normalized = self.whitespace_pattern.sub(' ', normalized)
        return normalized
    
    def _get_base_name(self, text: str) -> str:
        """Извлечение базового названия без номеров занятий/групп"""
        text = self.lesson_pattern.sub('', text)
        text = self.group_pattern.sub('', text)
        return text
    
    def matches_filter(self, event_name: str) -> bool:
        """Проверка соответствия названия мероприятия фильтрам"""
        normalized_name = self._normalize_text(event_name)
        base_name = self._get_base_name(normalized_name)
        
        return any(
            filter_name in base_name or base_name in filter_name
            for filter_name in self.normalized_filters
        )

def filter_courses(input_file: str, output_file: str, filter_names: List[str],
                  encoding: str = 'utf-8-sig', separator: str = ';') -> None:
    """
    Основная функция для фильтрации курсов
    
    Args:
        input_file: путь к входному файлу
        output_file: путь к выходному файлу  
        filter_names: список названий курсов для фильтрации
        encoding: кодировка файла
        separator: разделитель CSV
    """
    
    try:
        # Загружаем данные
        df = pd.read_csv(input_file, encoding=encoding, sep=separator)
        
        if "Название мероприятия" not in df.columns:
            print("Ошибка: колонка 'Название мероприятия' не найдена")
            return
        
        # Создаем и применяем фильтр
        course_filter = CourseFilter(filter_names)
        
        print(f"Исходное количество строк: {len(df)}")
        
        # Применяем фильтр
        filtered_df = df[df["Название мероприятия"].apply(course_filter.matches_filter)]
        
        print(f"Количество строк после фильтрации: {len(filtered_df)}")
        
        # Сохраняем результат
        filtered_df.to_csv(output_file, index=False, encoding=encoding, sep=separator)
        print(f"Отфильтрованный файл сохранен как: {output_file}")
        
    except FileNotFoundError:
        print(f"Ошибка: файл {input_file} не найден")
    except Exception as e:
        print(f"Ошибка при обработке: {e}")

def main():
    """Основная функция"""
    # Конфигурация
    INPUT_FILE = "reordered_file.csv"
    OUTPUT_FILE = "filtered_courses.csv"
    
    FILTER_NAMES = [
    "Oracle Database: Основы SQL","PostgreSQL: Уровень 1. Основы SQL","Разработка баз данных на платформе Microsoft SQL Server"
]
    
    # Запуск фильтрации
    filter_courses(INPUT_FILE, OUTPUT_FILE, FILTER_NAMES)

if __name__ == "__main__":
    main()

Исходное количество строк: 371
Количество строк после фильтрации: 178
Отфильтрованный файл сохранен как: filtered_courses.csv
